In [12]:
# relevant imports
from pydriller import Repository
from datetime import datetime, timedelta
import itertools
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from itertools import combinations, product
from collections import defaultdict

In [13]:
# Apache Kafka repo
path = "./kafka"

### Temporal Coupling

In [14]:
# Get all commits from repository
commits = Repository(path).traverse_commits()

In [5]:
files = set()

for commit in tqdm(commits, desc="Processing Commits", unit="commit"):
    modified_files = set([m.filename for m in commit.modified_files])
    files.update(modified_files)

Processing Commits: 11903commit [02:26, 81.32commit/s]


In [6]:
print(f"number of files: {len(files)}")

number of files: 7241


First create a commit Dataframe to look up commits with following columns:
- commit_id: int
- timestamp: date
- files: list of files

In [15]:
def create_commit_dataframe(path, commits):
    commit_dict = {}
    
    for commit in tqdm(commits, desc="Processing Commits", unit="commit"):
        commit_id = commit.hash
        commit_timestamp = commit.author_date
        modified_files = set([m.filename for m in commit.modified_files])

        commit_dict[commit_id] = {"timestamp": commit_timestamp,
                                  "files": modified_files}

    # Convert the dictionary to a DataFrame
    commit_df = pd.DataFrame.from_dict(commit_dict, orient='index').reset_index()
    commit_df.columns = ['commit_id', 'timestamp', 'files']

    return commit_df, commit_dict

commit_df, commit_dict = create_commit_dataframe(path, commits)

Processing Commits: 11903commit [02:28, 80.39commit/s] 


In [17]:
commit_df.head()

,commit_id,timestamp,files
0,642da2f28c9bc6e373603d6d9119ce33684090f5,2011-08-01 23:41:24+00:00,"{Pool.scala, encoder_helper_tests.cpp, MultiFe..."
1,cbbfbaf33179f2a44ae641ccaaba1f0a8e73730f,2011-08-03 17:34:03+00:00,{.gitignore}
2,665f1d0527846e1eaf6c39fd8a329dbfeb08b0c9,2011-08-05 01:36:43+00:00,{zkclient-20110412.jar}
3,96b8e03dd1ee23a550ca87803777b4d7bec05d5a,2011-08-07 16:29:55+00:00,"{ZKEphemeralTest.scala, ZookeeperConsumerConne..."
4,f7046c247dd466c90337bb39a4612115c3e41923,2011-08-07 23:56:05+00:00,{ProducerPool.scala}


Next loop through all commits, and check which other commits in the curren time_window searching the commit-dataframe.  
Adding/updating all file combinations of temporal coupled commits in dictionary.

In [18]:
TIME_WINDOWS = [24, 48, 72, 168]

In [20]:
def temporal_coupling(path, commit_df, commit_dict, time_windows, files):

    result_dict = {}
    commits = Repository(path).traverse_commits()


    for commit in tqdm(commit_dict, desc="Processing Commits", unit="commit"):

        commit_timestamp = commit_dict[commit]["timestamp"]
        modified_files = commit_dict[commit]["files"]
        modified_files = files & modified_files

        for time_window in time_windows:
            
            time_window_end = commit_timestamp + timedelta(hours=time_window)

            mask = True
            mask &= commit_df["timestamp"] >= commit_timestamp
            mask &= commit_df["timestamp"] <= time_window_end

            filtered_df = commit_df[mask]

            commits_in_window = set(commit_df["commit_id"].unique())
                
            modified_files_in_window = set(filtered_df['files'].explode().unique())
            modified_files_in_window = files & modified_files_in_window

            result_dict = add_coupled_files_to_dict(result_dict, modified_files, modified_files_in_window, time_window, files)

    return result_dict

def add_coupled_files_to_dict(dictionary, file_set_1, file_set_2, time_window, files):

    file_combinations = [(file1, file2) for file1, file2 in product(file_set_1, file_set_2) if file1 != file2]

    for file1, file2 in file_combinations:
        combination = sorted([file1, file2])
        key = f"{combination[0]} - {combination[1]}"

        if key not in dictionary:
            dictionary[key] = {
                "files": [file1, file2],
                "time_windows": {24: 0, 48: 0, 72: 0, 168: 0}
            }

        dictionary[key]["time_windows"][time_window] = dictionary[key]["time_windows"].get(time_window, 0) + 1
   
    return dictionary


In [21]:
temporal_coupling_result = temporal_coupling(path, commit_df, commit_dict, TIME_WINDOWS, files)

Processing Commits: 100%|█████████████| 11903/11903 [23:02<00:00,  8.61commit/s]


In [28]:
def store_temporal_coupling_to_json(dictionary, file_name):

    result_list = []

    for key in tqdm(dictionary, desc="Processing Coupled Files", unit="file-pair"):

        file_pair = dictionary[key]["files"]
        joint_commits_24 = dictionary[key]["time_windows"][24]
        joint_commits_48 = dictionary[key]["time_windows"][48]
        joint_commits_72 = dictionary[key]["time_windows"][72]
        joint_commits_168 = dictionary[key]["time_windows"][168]


        # Create a dictionary for the current pair
        current_dict = {
            "file_pair": file_pair,
            "coupled_commits": [
                {
                    "time_window": 24,
                    "commit_count": joint_commits_24
                },
                {
                    "time_window": 48,
                    "commit_count": joint_commits_48
                },
                {
                    "time_window": 72,
                    "commit_count": joint_commits_72
                },
                {
                    "time_window": 168,
                    "commit_count": joint_commits_168
                },
            ]
        }

        # Append the dictionary to the result list
        result_list.append(current_dict)

    # Save the list of dictionaries to a JSON file
    with open(file_name, 'w') as json_file:
        json.dump(result_list, json_file, indent=2)

In [29]:
store_temporal_coupling_to_json(temporal_coupling_result, "temporal_coupling.JSON")

Processing Coupled Files: 100%|█| 4705414/4705414 [00:18<00:00, 252753.67file-pa


In [26]:
def find_top_entries_with_largest_temporal_coupling(comb_dict, time_window, top_n):
    top_entries = sorted(comb_dict.items(), key=lambda x: x[1]["time_windows"][time_window], reverse=True)[:top_n]
    
    return top_entries


10 Strongest temporal coupled files in 24 h window:

In [27]:
find_top_entries_with_largest_temporal_coupling(temporal_coupling_result, 24, 10)

[('KafkaApis.scala - ReplicaManager.scala',
  {'files': ['KafkaApis.scala', 'ReplicaManager.scala'],
   'time_windows': {24: 397, 48: 483, 72: 521, 168: 675}}),
 ('StreamThread.java - StreamThreadTest.java',
  {'files': ['StreamThread.java', 'StreamThreadTest.java'],
   'time_windows': {24: 370, 48: 388, 72: 402, 168: 448}}),
 ('KafkaApis.scala - RequestResponseTest.java',
  {'files': ['KafkaApis.scala', 'RequestResponseTest.java'],
   'time_windows': {24: 369, 48: 411, 72: 433, 168: 508}}),
 ('Partition.scala - ReplicaManager.scala',
  {'files': ['Partition.scala', 'ReplicaManager.scala'],
   'time_windows': {24: 353, 48: 375, 72: 393, 168: 439}}),
 ('KafkaApis.scala - TestUtils.scala',
  {'files': ['TestUtils.scala', 'KafkaApis.scala'],
   'time_windows': {24: 347, 48: 420, 72: 473, 168: 641}}),
 ('Log.scala - LogTest.scala',
  {'files': ['LogTest.scala', 'Log.scala'],
   'time_windows': {24: 341, 48: 357, 72: 370, 168: 410}}),
 ('KafkaApis.scala - KafkaServer.scala',
  {'files': ['K

10 Strongest temporal coupled files in 48 h window:

In [23]:
find_top_entries_with_largest_temporal_coupling(temporal_coupling_result, 48, 10)

[('KafkaApis.scala - ReplicaManager.scala',
  {'files': ['KafkaApis.scala', 'ReplicaManager.scala'],
   'time_windows': {24: 397, 48: 483, 72: 521, 168: 675}}),
 ('KafkaApis.scala - TestUtils.scala',
  {'files': ['TestUtils.scala', 'KafkaApis.scala'],
   'time_windows': {24: 347, 48: 420, 72: 473, 168: 641}}),
 ('KafkaApis.scala - RequestResponseTest.java',
  {'files': ['KafkaApis.scala', 'RequestResponseTest.java'],
   'time_windows': {24: 369, 48: 411, 72: 433, 168: 508}}),
 ('KafkaApis.scala - KafkaServer.scala',
  {'files': ['KafkaServer.scala', 'KafkaApis.scala'],
   'time_windows': {24: 332, 48: 396, 72: 433, 168: 573}}),
 ('StreamThread.java - StreamThreadTest.java',
  {'files': ['StreamThread.java', 'StreamThreadTest.java'],
   'time_windows': {24: 370, 48: 388, 72: 402, 168: 448}}),
 ('build.gradle - dependencies.gradle',
  {'files': ['build.gradle', 'dependencies.gradle'],
   'time_windows': {24: 330, 48: 379, 72: 409, 168: 529}}),
 ('Partition.scala - ReplicaManager.scala',


10 Strongest temporal coupled files in 72 h window:

In [24]:
find_top_entries_with_largest_temporal_coupling(temporal_coupling_result, 72, 10)

[('KafkaApis.scala - ReplicaManager.scala',
  {'files': ['KafkaApis.scala', 'ReplicaManager.scala'],
   'time_windows': {24: 397, 48: 483, 72: 521, 168: 675}}),
 ('KafkaApis.scala - TestUtils.scala',
  {'files': ['TestUtils.scala', 'KafkaApis.scala'],
   'time_windows': {24: 347, 48: 420, 72: 473, 168: 641}}),
 ('KafkaApis.scala - KafkaServer.scala',
  {'files': ['KafkaServer.scala', 'KafkaApis.scala'],
   'time_windows': {24: 332, 48: 396, 72: 433, 168: 573}}),
 ('KafkaApis.scala - RequestResponseTest.java',
  {'files': ['KafkaApis.scala', 'RequestResponseTest.java'],
   'time_windows': {24: 369, 48: 411, 72: 433, 168: 508}}),
 ('KafkaApis.scala - build.gradle',
  {'files': ['KafkaApis.scala', 'build.gradle'],
   'time_windows': {24: 233, 48: 344, 72: 427, 168: 672}}),
 ('KafkaApis.scala - KafkaConfig.scala',
  {'files': ['KafkaConfig.scala', 'KafkaApis.scala'],
   'time_windows': {24: 282, 48: 363, 72: 418, 168: 589}}),
 ('build.gradle - dependencies.gradle',
  {'files': ['build.grad

10 Strongest temporal coupled files in 168 h window:

In [25]:
find_top_entries_with_largest_temporal_coupling(temporal_coupling_result, 168, 10)

[('KafkaApis.scala - ReplicaManager.scala',
  {'files': ['KafkaApis.scala', 'ReplicaManager.scala'],
   'time_windows': {24: 397, 48: 483, 72: 521, 168: 675}}),
 ('KafkaApis.scala - build.gradle',
  {'files': ['KafkaApis.scala', 'build.gradle'],
   'time_windows': {24: 233, 48: 344, 72: 427, 168: 672}}),
 ('KafkaApis.scala - TestUtils.scala',
  {'files': ['TestUtils.scala', 'KafkaApis.scala'],
   'time_windows': {24: 347, 48: 420, 72: 473, 168: 641}}),
 ('KafkaApis.scala - KafkaConfig.scala',
  {'files': ['KafkaConfig.scala', 'KafkaApis.scala'],
   'time_windows': {24: 282, 48: 363, 72: 418, 168: 589}}),
 ('KafkaApis.scala - KafkaServer.scala',
  {'files': ['KafkaServer.scala', 'KafkaApis.scala'],
   'time_windows': {24: 332, 48: 396, 72: 433, 168: 573}}),
 ('KafkaApis.scala - Log.scala',
  {'files': ['Log.scala', 'KafkaApis.scala'],
   'time_windows': {24: 281, 48: 342, 72: 394, 168: 543}}),
 ('build.gradle - dependencies.gradle',
  {'files': ['build.gradle', 'dependencies.gradle'],
 

### Logical Coupling

The function logical_coupling is the main function which computes the Logical-Coupling between all files in the repository.  
Store_coupled_files_to_dict and store_file_to_dict are just helper functions to store the data.  

In [7]:
def logical_coupling(url_repo):

    combination_dict = {}
    individual_dict = {}

    commits = Repository(url_repo).traverse_commits()
    
    for commit in tqdm(commits, desc="Processing Commits", unit="commit"):

        modified_files = set([m.filename for m in commit.modified_files])

        if len(modified_files) < 2: #no coupled files in commit
            continue
        
        file_combinations = list(itertools.combinations(modified_files, 2))

        for combination in file_combinations:

            combination_dict = store_coupled_files_in_dict(combination_dict, combination)
            
        individual_dict = store_file_in_dict(individual_dict, modified_files)
            
        #print("-"*20)

    return combination_dict, individual_dict

def store_coupled_files_in_dict(dictionary, combination):
    
    combination = sorted(combination)
    key = combination[0] + " - " + combination[1]

    if key not in dictionary:
        dictionary[key] = 1
        dictionary[key] = {
                        "files": combination,
                        "joint_commits": 1
                    }
        
    else:
        dictionary[key]["joint_commits"]+= 1

    return dictionary

def store_file_in_dict(dictionary, files):

    for file in files:

        if file not in dictionary:
            dictionary[file] = 1

        else:
            dictionary[file]+= 1

    return dictionary

In [8]:
def store_logical_coupling_to_json(comb_dict, ind_dict, file_name):

    result_list = []

    # Create a tqdm progress bar for the loop
    for key in tqdm(comb_dict, desc="Writing to JSON", unit=" pair"):

        file_pair = comb_dict[key]["files"]
        joint_commits = comb_dict[key]["joint_commits"]

        file1_commits = ind_dict[file_pair[0]] - joint_commits
        file2_commits = ind_dict[file_pair[1]] - joint_commits

        # Create a dictionary for the current pair
        current_dict = {
            "file_pair": file_pair,
            "logical_coupling": {
                "Joint": joint_commits,
                file_pair[0]: file1_commits,
                file_pair[1]: file2_commits
            }
        }

        # Append the dictionary to the result list
        result_list.append(current_dict)

    # Save the list of dictionaries to a JSON file
    with open(file_name, 'w') as json_file:
        json.dump(result_list, json_file, indent=2)

Compute Logical Coupling over entire Repository

In [9]:
comb_files_result, ind_files_result = logical_coupling(path)

Processing Commits: 11903commit [02:37, 75.81commit/s]


Store Logical Coupling result to JSON file.

In [11]:
store_logical_coupling_to_json(comb_files_result, ind_files_result, "logical_coupling.JSON")

Writing to JSON: 100%|█████████| 1258215/1258215 [00:03<00:00, 404415.55 pair/s]


Find the top 10 files which are tightest logical coupled

In [14]:
def find_top_entries_with_largest_joint_commits(comb_dict, top_n=3):
    top_entries = sorted(comb_dict.items(), key=lambda x: x[1]["joint_commits"], reverse=True)[:top_n]
    
    return top_entries

In [15]:
find_top_entries_with_largest_joint_commits(comb_files_result, 10)

[('StreamThread.java - StreamThreadTest.java',
  {'files': ['StreamThread.java', 'StreamThreadTest.java'],
   'joint_commits': 170}),
 ('Log.scala - LogTest.scala',
  {'files': ['Log.scala', 'LogTest.scala'], 'joint_commits': 159}),
 ('Partition.scala - ReplicaManager.scala',
  {'files': ['Partition.scala', 'ReplicaManager.scala'],
   'joint_commits': 159}),
 ('KafkaApis.scala - RequestResponseTest.java',
  {'files': ['KafkaApis.scala', 'RequestResponseTest.java'],
   'joint_commits': 156}),
 ('KafkaApis.scala - ReplicaManager.scala',
  {'files': ['KafkaApis.scala', 'ReplicaManager.scala'],
   'joint_commits': 148}),
 ('KafkaApis.scala - KafkaApisTest.scala',
  {'files': ['KafkaApis.scala', 'KafkaApisTest.scala'], 'joint_commits': 146}),
 ('ReplicaManager.scala - ReplicaManagerTest.scala',
  {'files': ['ReplicaManager.scala', 'ReplicaManagerTest.scala'],
   'joint_commits': 141}),
 ('StreamTask.java - StreamTaskTest.java',
  {'files': ['StreamTask.java', 'StreamTaskTest.java'], 'joint_

#### In-depth analysis of 3 sets of strongly coupled files

We look at 3 sets of files, which are strongly coupled by **logical coupling**:

##### 1. `StreamThread.java` and `StreamThreadTest.java`

With 170 joint commits StreamThread.java and StreamThreadTest.java are strongest coupled files from the whole Kafka repository.
It is reasonable that when modifications in the class of StreamThread.java are made, that also the tests need to adjusted accordingly in the StreamThreadTest.java . It makes sense, that there is always a coupling between a file and it's corresponding test-file, they go often hand-in-hand. --> Unit-Test-Relationship

##### 2. `Partition.scala`and `ReplicaManager.scala`

Tese two files have 159 joint commits and also strongly logical coupled.  
The files have an architectural dependency. Meaning that ReplicaManager is responsible for managing the state of partitions. 
To manage the states of partitions, ReplicaManager.scala is dependend on Partition.scala, which means modifications in Partition.scala often lead to modifications in ReplicaManager.scala.

##### 3. `KafkaApis.scala`and `ReplicaManager.scala`

These two filese have 148 joint commits. These files have also an architectural dependency.  
Meaning KafkaApi.scala seem to strongly depend on ReplicaManager.scala, resulting that modifications in ReplicaManager.scale enforce also changes in KafkaApis.scala.